In [1]:
# CSL Paper: Dimensional speech emotion recognition from acoustic and text
# Changelog:
# 2019-09-01: initial version
# 2019-10-06: optimizer MTL parameters with linear search (in progress)
# 2019-12-21: update file for CSL journal
#             this python use following feature: mean, std, silence

import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#from keras_self_attention import SeqSelfAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from attention_helper import AttentionDecoder
#from read_csv import load_features

import random as rn
import tensorflow as tf

# import gensim
# from gensim.models import Word2Vec
# from gensim.models.keyedvectors import KeyedVectors
import codecs

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# x_text = np.load('../improv_loso/data/x_text.npy')

In [4]:
# x_text.shape

In [5]:
# load data - label
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_train = pd.read_csv(list_path_train, index_col=None)

# vad_train = [list_train['v'], list_train['a'], list_train['d']]
# vad_train = np.array(vad_train).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

## Text processing train
list_file = pd.concat([list_train, list_test])
text = list_file['transcript']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
nb_words = len(word_index) + 1
print('Found %s unique tokens' % len(word_index))

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_text = []
x_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# x_text = np.load('improv_loso/data/x_text.npy')
# x_train_text = x_text[:5744] 
# x_test_text = x_text[5744:]

# vad = np.load('improv_loso/data/vad.npy')
# vad_train = vad[:5744]
# vad_test = vad[5744:]

EMBEDDING_DIM = 300
# nb_words = 3438
# MAX_SEQUENCE_LENGTH = 300

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
file_loc =  '../../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

Found 3427 unique tokens
../../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 167


In [6]:
vad = np.load('../improv_loso/data/vad.npy')

In [7]:
# standardization
scaled_feature = False

# set Dropout
do = 0.3

# standardized label
scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad


# Concordance correlation coefficient (CCC)-based loss function
# using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [8]:
# split train/test
split = 5744
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)

In [9]:
# API model, if use RNN, first two rnn layer must return_sequences=True
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
#                     weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model1 = text_model1()

In [10]:
# main function for for LOSO
def main(time):
    model_1 = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [x_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [x_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict(x_text[4596:], batch_size=32)
    np.save('5744npy-WE/text_WE5744-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 2570).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('5744npy-WE/text_WE-5744.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 93s 20ms/step - loss: 2.5534 - v_loss: 0.7322 - a_loss: 0.9005 - d_loss: 0.9163 - v_ccc: 0.2663 - a_ccc: 0.0979 - d_ccc: 0.0824 - val_loss: 2.6463 - val_v_loss: 0.8314 - val_a_loss: 0.9139 - val_d_loss: 0.9002 - val_v_ccc: 0.1685 - val_a_ccc: 0.0859 - val_d_ccc: 0.0993
Epoch 2/50
4595/4595 [==============================] - 87s 19ms/step - loss: 1.9842 - v_loss: 0.5535 - a_loss: 0.6962 - d_loss: 0.7349 - v_ccc: 0.4464 - a_ccc: 0.3042 - d_ccc: 0.2652 - val_loss: 2.6758 - val_v_loss: 0.8482 - val_a_loss: 0.9025 - val_d_loss: 0.9243 - val_v_ccc: 0.1516 - val_a_ccc: 0.0975 - val_d_ccc: 0.0751
Epoch 3/50
4595/4595 [==============================] - 88s 19ms/step - loss: 1.7401 - v_loss: 0.4816 - a_loss: 0.6068 - d_loss: 0.6517 - v_ccc: 0.5182 - a_ccc: 0.3932 - d_ccc: 0.3485 - val_loss: 2.6797 - val_v_loss: 0

Epoch 13/50
4595/4595 [==============================] - 88s 19ms/step - loss: 0.8531 - v_loss: 0.2374 - a_loss: 0.3043 - d_loss: 0.3104 - v_ccc: 0.7623 - a_ccc: 0.6952 - d_ccc: 0.6894 - val_loss: 2.7064 - val_v_loss: 0.8747 - val_a_loss: 0.9046 - val_d_loss: 0.9263 - val_v_ccc: 0.1247 - val_a_ccc: 0.0951 - val_d_ccc: 0.0737
Epoch 14/50
1422/1422 [==============================] - 8s 5ms/step
[0.21117457747459412, 0.17811916768550873, 0.13577933609485626]
1
Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 90s 20ms/step - loss: 2.6073 - v_loss: 0.7215 - a_loss: 0.9400 - d_loss: 0.9446 - v_ccc: 0.2786 - a_ccc: 0.0594 - d_ccc: 0.0548 - val_loss: 2.6428 - val_v_loss: 0.8376 - val_a_loss: 0.9024 - val_d_loss: 0.9014 - val_v_ccc: 0.1618 - val_a_ccc: 0.0974 - val_d_ccc: 0.0980
Epoch 2/50
4595/4595 [==============================] - 87s 19ms/step - loss: 2.0200 - v_loss: 0.5444 - a_loss: 0.7164 - d_loss: 0.7577 - v_ccc: 0.4550 - a_ccc: 0.2

4595/4595 [==============================] - 88s 19ms/step - loss: 0.9006 - v_loss: 0.2484 - a_loss: 0.3195 - d_loss: 0.3326 - v_ccc: 0.7519 - a_ccc: 0.6801 - d_ccc: 0.6674 - val_loss: 2.7858 - val_v_loss: 0.8771 - val_a_loss: 0.9241 - val_d_loss: 0.9842 - val_v_ccc: 0.1226 - val_a_ccc: 0.0759 - val_d_ccc: 0.0157
Epoch 13/50
1422/1422 [==============================] - 8s 5ms/step
[0.20563235878944397, 0.14440056681632996, 0.14071963727474213]
3
Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 89s 19ms/step - loss: 2.5613 - v_loss: 0.7484 - a_loss: 0.8988 - d_loss: 0.9128 - v_ccc: 0.2511 - a_ccc: 0.1004 - d_ccc: 0.0873 - val_loss: 2.6718 - val_v_loss: 0.8244 - val_a_loss: 0.9160 - val_d_loss: 0.9301 - val_v_ccc: 0.1754 - val_a_ccc: 0.0838 - val_d_ccc: 0.0689
Epoch 2/50
4595/4595 [==============================] - 88s 19ms/step - loss: 2.0016 - v_loss: 0.5589 - a_loss: 0.7072 - d_loss: 0.7346 - v_ccc: 0.4410 - a_ccc: 0.2924 - d_ccc:

1422/1422 [==============================] - 8s 6ms/step
[0.2030608355998993, 0.23502853512763977, 0.1741943657398224]
5
Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 91s 20ms/step - loss: 2.6133 - v_loss: 0.7669 - a_loss: 0.9025 - d_loss: 0.9426 - v_ccc: 0.2324 - a_ccc: 0.0967 - d_ccc: 0.0576 - val_loss: 2.8365 - val_v_loss: 0.8381 - val_a_loss: 0.9783 - val_d_loss: 1.0201 - val_v_ccc: 0.1619 - val_a_ccc: 0.0217 - val_d_ccc: -0.0201
Epoch 2/50
4595/4595 [==============================] - 88s 19ms/step - loss: 2.0048 - v_loss: 0.5647 - a_loss: 0.6989 - d_loss: 0.7412 - v_ccc: 0.4351 - a_ccc: 0.3011 - d_ccc: 0.2590 - val_loss: 2.7086 - val_v_loss: 0.8296 - val_a_loss: 0.9356 - val_d_loss: 0.9434 - val_v_ccc: 0.1703 - val_a_ccc: 0.0645 - val_d_ccc: 0.0566
Epoch 3/50
4595/4595 [==============================] - 88s 19ms/step - loss: 1.7420 - v_loss: 0.4921 - a_loss: 0.5952 - d_loss: 0.6550 - v_ccc: 0.5080 - a_ccc: 0.4049 - d_ccc: 0

4595/4595 [==============================] - 88s 19ms/step - loss: 0.9530 - v_loss: 0.2614 - a_loss: 0.3424 - d_loss: 0.3492 - v_ccc: 0.7384 - a_ccc: 0.6579 - d_ccc: 0.6507 - val_loss: 2.8368 - val_v_loss: 0.8869 - val_a_loss: 0.9534 - val_d_loss: 0.9966 - val_v_ccc: 0.1128 - val_a_ccc: 0.0468 - val_d_ccc: 0.0036
Epoch 12/50
1422/1422 [==============================] - 8s 6ms/step
[0.22640632092952728, 0.2000390738248825, 0.18405979871749878]
7
Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 92s 20ms/step - loss: 2.6593 - v_loss: 0.7617 - a_loss: 0.9248 - d_loss: 0.9727 - v_ccc: 0.2372 - a_ccc: 0.0757 - d_ccc: 0.0278 - val_loss: 2.6833 - val_v_loss: 0.8389 - val_a_loss: 0.8975 - val_d_loss: 0.9456 - val_v_ccc: 0.1608 - val_a_ccc: 0.1021 - val_d_ccc: 0.0538
Epoch 2/50
4595/4595 [==============================] - 84s 18ms/step - loss: 2.0344 - v_loss: 0.5599 - a_loss: 0.7064 - d_loss: 0.7672 - v_ccc: 0.4399 - a_ccc: 0.2932 - d_ccc: 

...

4595/4595 [==============================] - 88s 19ms/step - loss: 1.0125 - v_loss: 0.2743 - a_loss: 0.3638 - d_loss: 0.3742 - v_ccc: 0.7257 - a_ccc: 0.6363 - d_ccc: 0.6255 - val_loss: 2.7743 - val_v_loss: 0.8731 - val_a_loss: 0.9217 - val_d_loss: 0.9789 - val_v_ccc: 0.1265 - val_a_ccc: 0.0782 - val_d_ccc: 0.0210
Epoch 11/50
4595/4595 [==============================] - 88s 19ms/step - loss: 0.9705 - v_loss: 0.2649 - a_loss: 0.3444 - d_loss: 0.3626 - v_ccc: 0.7355 - a_ccc: 0.6559 - d_ccc: 0.6381 - val_loss: 2.8072 - val_v_loss: 0.8775 - val_a_loss: 0.9314 - val_d_loss: 0.9970 - val_v_ccc: 0.1219 - val_a_ccc: 0.0682 - val_d_ccc: 0.0027
Epoch 12/50
1422/1422 [==============================] - 8s 6ms/step
[0.18061478435993195, 0.14595098793506622, 0.17133402824401855]
11
Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 93s 20ms/step - loss: 2.5845 - v_loss: 0.7388 - a_loss: 0.9262 - d_loss: 0.9184 - v_ccc: 0.2608 - a_ccc: 0.0729 - d_cc

4595/4595 [==============================] - 90s 20ms/step - loss: 1.0053 - v_loss: 0.2771 - a_loss: 0.3547 - d_loss: 0.3746 - v_ccc: 0.7226 - a_ccc: 0.6464 - d_ccc: 0.6257 - val_loss: 2.7977 - val_v_loss: 0.8810 - val_a_loss: 0.9352 - val_d_loss: 0.9804 - val_v_ccc: 0.1185 - val_a_ccc: 0.0645 - val_d_ccc: 0.0193
Epoch 11/50
4595/4595 [==============================] - 91s 20ms/step - loss: 0.9435 - v_loss: 0.2599 - a_loss: 0.3344 - d_loss: 0.3492 - v_ccc: 0.7399 - a_ccc: 0.6658 - d_ccc: 0.6509 - val_loss: 2.8129 - val_v_loss: 0.8843 - val_a_loss: 0.9366 - val_d_loss: 0.9916 - val_v_ccc: 0.1152 - val_a_ccc: 0.0634 - val_d_ccc: 0.0084
Epoch 12/50
4595/4595 [==============================] - 90s 20ms/step - loss: 0.8982 - v_loss: 0.2498 - a_loss: 0.3141 - d_loss: 0.3336 - v_ccc: 0.7499 - a_ccc: 0.6857 - d_ccc: 0.6662 - val_loss: 2.7692 - val_v_loss: 0.8783 - val_a_loss: 0.9170 - val_d_loss: 0.9731 - val_v_ccc: 0.1213 - val_a_ccc: 0.0828 - val_d_ccc: 0.0267
Epoch 13/50
1422/1422 [========

4595/4595 [==============================] - 91s 20ms/step - loss: 1.0232 - v_loss: 0.2815 - a_loss: 0.3658 - d_loss: 0.3759 - v_ccc: 0.7184 - a_ccc: 0.6341 - d_ccc: 0.6243 - val_loss: 2.7670 - val_v_loss: 0.8806 - val_a_loss: 0.9206 - val_d_loss: 0.9657 - val_v_ccc: 0.1190 - val_a_ccc: 0.0795 - val_d_ccc: 0.0346
Epoch 11/50
1422/1422 [==============================] - 8s 6ms/step
[0.1606437861919403, 0.2197180688381195, 0.19582252204418182]
15
Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 97s 21ms/step - loss: 2.5540 - v_loss: 0.7441 - a_loss: 0.8926 - d_loss: 0.9169 - v_ccc: 0.2557 - a_ccc: 0.1070 - d_ccc: 0.0833 - val_loss: 2.7122 - val_v_loss: 0.8340 - val_a_loss: 0.9305 - val_d_loss: 0.9471 - val_v_ccc: 0.1660 - val_a_ccc: 0.0695 - val_d_ccc: 0.0524
Epoch 2/50
4595/4595 [==============================] - 90s 20ms/step - loss: 1.9806 - v_loss: 0.5493 - a_loss: 0.6835 - d_loss: 0.7483 - v_ccc: 0.4503 - a_ccc: 0.3170 - d_ccc: 

4595/4595 [==============================] - 89s 19ms/step - loss: 0.9442 - v_loss: 0.2593 - a_loss: 0.3346 - d_loss: 0.3501 - v_ccc: 0.7405 - a_ccc: 0.6655 - d_ccc: 0.6498 - val_loss: 2.7759 - val_v_loss: 0.8620 - val_a_loss: 0.9324 - val_d_loss: 0.9811 - val_v_ccc: 0.1377 - val_a_ccc: 0.0676 - val_d_ccc: 0.0188
Epoch 12/50
4595/4595 [==============================] - 89s 19ms/step - loss: 0.9012 - v_loss: 0.2550 - a_loss: 0.3173 - d_loss: 0.3299 - v_ccc: 0.7453 - a_ccc: 0.6837 - d_ccc: 0.6699 - val_loss: 2.7559 - val_v_loss: 0.8730 - val_a_loss: 0.9245 - val_d_loss: 0.9577 - val_v_ccc: 0.1266 - val_a_ccc: 0.0754 - val_d_ccc: 0.0420
Epoch 13/50
1422/1422 [==============================] - 8s 5ms/step
[0.20486482977867126, 0.1730317324399948, 0.10183810442686081]
17
Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 96s 21ms/step - loss: 2.5965 - v_loss: 0.7516 - a_loss: 0.9209 - d_loss: 0.9229 - v_ccc: 0.2491 - a_ccc: 0.0783 - d_ccc

1422/1422 [==============================] - 8s 6ms/step
[0.18430666625499725, 0.19586041569709778, 0.11789774149656296]
19


In [11]:
# hist1 = model1.fit(x_text[:split], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate(x_text[split:], vad[split:].T.tolist())
# print(eval_metrik1[-3:])

# # make prediction
# predict = model1.predict(x_text[4596:], batch_size=32)
# np.save('text_glove_npy_loso', 
#          np.array(predict).reshape(3, 2570).T)